In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import lightgbm as lgb
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score


In [ ]:
data = pd.read_csv('/kaggle/input/playground-series-s4e5/train.csv')
data.head()


In [ ]:
Y = data[["FloodProbability"]]
X = data
X = X.drop(['id','FloodProbability'],axis=1)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
parameters = {
    'max_depth': [3, 5, 7],
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
}

In [ ]:
#model = lgb.LGBMRegressor(objective='regression', num_leaves=15, learning_rate=0.05, n_estimators=100)
model = xgb.XGBRegressor(objective ='reg:squarederror', random_state=42)


In [ ]:
# Setup GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=parameters, scoring='r2', cv=5, verbose=1)

# Fit GridSearchCV
grid_search.fit(X_train, Y_train)


In [ ]:
print(f"Best Score: {grid_search.best_score_}")

In [ ]:
print("Best parameters set")
best_params = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print(f"\t{param_name}: {best_params[param_name]}")

In [ ]:
best_model = grid_search.best_estimator_

In [ ]:
prediction = best_model.predict(X_test)

In [ ]:
r2_score(Y_test,prediction)

## Submission

In [ ]:
test_data = pd.read_csv('/kaggle/input/playground-series-s4e5/test.csv')

In [ ]:
test_ids = test_data['id']

In [ ]:
X_sub_test = test_data.drop(columns=['id'])

In [ ]:
sub_preds = best_model.predict(X_sub_test)

In [ ]:
submission = pd.DataFrame({
    'id': test_ids,
    'FloodProbability': sub_preds
})

In [ ]:
submission.to_csv('xgb_submission.csv', index=False)
